In [1]:
import os
import re
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import ast
import json
from glob import glob

In [2]:
#padded_image_shape = (1024, 1920)
min_stride = 128
padded_image_shape = (128*4, 128*8)
anchor_k = 9
num_classes = 80#
max_data_m = 50#00
edgecolors = np.random.rand(num_classes, 3) 
edgecolors = np.minimum(edgecolors+0.1, 1.0)
#class_names = ['bg', 'pig', 'crow']
class_names_kor = ['', '모름', '돼지', '멧돼지', '큰부리까마귀', '흰배지빠귀', '유리딱새', '동고비', 
               '박새', '호랑지빠귀', '검독수리', '너구리', '설치류', '다람쥐', '사람']
class_names = ['bg','unknown', 'pig', 'boar', 'crow', 'bird_hinbejipagi', 'bird_ddaksae', 
               'bird_dongobi', 'bird_baksae', 'bird_horangjipagi', 'bird_eagle', 
               'racoon', 'rat', 'squrrel', 'human']

folder_pig = 'D:/dataset/pig/'
os.path.isdir(folder_pig)

True

In [3]:
''.endswith

<function str.endswith>

In [4]:
def add_folder_to_file_name(folder_pig, path_csv):
    path_csv = folder_pig + path_csv
    path_weight = "retinanet/pig_efficient_det_BiFPN_D3"
    print(os.path.isdir(folder_pig), os.path.isfile(path_csv))
    if os.path.isfile(path_csv):
        
        df = pd.read_csv(path_csv)
        df['filename'] = folder_pig + df['filename']
        print('df', df.shape, df['filename'].nunique(), df['region_attributes'].nunique()  )
        print(df['region_attributes'].unique())
        return df
    else:
        return None


In [5]:
def read_df(folder_pig, path_csv):
    path_csv = folder_pig + path_csv
    if path_csv.endswith('xlsx'):
        df = pd.read_excel(path_csv)
    else:
        df = pd.read_csv(path_csv)
    
    print(df.shape)
    return df

In [6]:
#path_images = glob(folder_pig + 'pig2-2020-11/*/*/*.jpg')
path_images = glob(folder_pig + 'geomex-pig05/*/*/*.jpg')
#path_images = glob(folder_pig + 'pig6-2020-11/*/*/*.jpg')
#path_images = glob(folder_pig + 'object3/*.jpg')
len(path_images), path_images[0]

(73459,
 'D:/dataset/pig/geomex-pig05\\2020-09-02\\image\\geomex-pig05_2020-09-02_121552_temp-22.00_wat-87.20.jpg')

In [8]:
df = read_df(folder_pig, 'annotation/test5-202012-1.csv' )

(2023, 7)


In [9]:
df['region_attributes'].unique()

array(['{}', '{"class":"모름"}', '{"class":"흰배지빠귀"}', '{"class":"유리딱새"}',
       '{"class":"박새"}', '{"class":"큰부리까마귀"}', '{"class":undefined}'],
      dtype=object)

In [20]:
df[df['region_attributes']=='{"class":undefined}']

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
1124,geomex-pig05_2020-12-01_155818_temp-2.10_wat-3...,360334,{},1,0,"{""name"":""rect"",""x"":655,""y"":473,""width"":18,""hei...","{""class"":undefined}"


In [10]:
df

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,geomex-pig05_2020-12-07_000056_temp-1.50_wat-9...,72362,{},0,0,{},{}
1,geomex-pig05_2020-12-07_000300_temp-1.40_wat-9...,71983,{},0,0,{},{}
2,geomex-pig05_2020-12-07_000825_temp-1.40_wat-9...,78608,{},0,0,{},{}
3,geomex-pig05_2020-12-07_001029_temp-1.30_wat-9...,71870,{},0,0,{},{}
4,geomex-pig05_2020-12-07_001234_temp-1.30_wat-9...,78994,{},0,0,{},{}
...,...,...,...,...,...,...,...
2018,geomex-pig05_2020-12-02_235101_temp--1.50_wat-...,72184,{},0,0,{},{}
2019,geomex-pig05_2020-12-02_235306_temp--1.50_wat-...,72236,{},0,0,{},{}
2020,geomex-pig05_2020-12-02_235511_temp--1.50_wat-...,78873,{},0,0,{},{}
2021,geomex-pig05_2020-12-02_235716_temp--1.50_wat-...,72205,{},0,0,{},{}


In [11]:
annotation = dict()
object_width_list = []
for i in range(len(df)):
    
    sample_row = df.iloc[i]
    filename = sample_row['filename']    
    region_id = sample_row['region_id']
    box = sample_row['region_shape_attributes']    
    region_attributes = sample_row['region_attributes'] 
    if '(' in filename:
        print('()', filename)
    try:
        box_map = ast.literal_eval(box)    
        region_attributes = ast.literal_eval(region_attributes)        
    except:
        print('except filename',i, filename)
        print('box', box)
        print('region_attributes', region_attributes)
        continue
    
    if len(box_map.keys())<1 or len(region_attributes.keys())<1 :
        #print(i, 'no_keys_skip', filename.split(os.sep)[-1])
        continue
        
    cls = region_attributes['class']
    if cls in class_names_kor:
        cls_num = class_names_kor.index(cls)
    elif cls in class_names:
        cls_num = class_names.index(cls)
    else:
        print('cls unknown', cls)
        print('filename', filename)
        continue
        #xit()
        
    x0 = box_map['x']
    y0 = box_map['y']
    width = box_map['width']
    height = box_map['height']    
   
    object_width_list.append(width)    
    bbox = [cls_num, x0, y0, x0 + width, y0 + height]
    
    if filename not in annotation.keys():
        annotation[filename] = []
    
    annotation[filename].append(bbox)            
    #print(i, df.loc[i, 'filename'], width, height)

except filename 1124 geomex-pig05_2020-12-01_155818_temp-2.10_wat-37.10.jpg
box {"name":"rect","x":655,"y":473,"width":18,"height":19}
region_attributes {"class":undefined}


In [12]:
len(annotation)

74

In [13]:
def load_pig():
    input_list = []
    bbox_list = []
    path_list = []
    for path_image in annotation:
        cls_bbox = annotation[path_image]
        cls_bbox = np.array(cls_bbox).reshape([-1, 5])
        cls = cls_bbox[:, 0:1]
        bbox = np.array(cls_bbox[:, 1:])
        
        #print('path_image', path_image)
        if True:
            for full_path in path_images:
                if path_image in full_path:
                    path_image = full_path

        if os.path.isfile(path_image):
            img = Image.open(path_image)    
            #img = img.resize((1000,500))
           
            scale = np.array((img.width, img.height, img.width, img.height))
            scale = np.reshape(scale, (1, 4))       
            #print(key, cls, cls_bbox.dtype, cls_bbox, 'wh',img.width, img.height)
            img_arr = np.array(img)        
            del img
            bbox_norm = bbox.astype(np.float) / scale.astype(np.float)
            cls = np.array(cls).reshape((-1, 1))
            cls_bbox_norm = np.concatenate((cls, bbox_norm), axis=1)
                
            input_list.append(img_arr)
            bbox_list.append(cls_bbox_norm)
            path_list.append(full_path)
            if len(input_list)%100==0:        
                print(len(annotation), len(input_list), len(bbox_list))   
        else:
            print('not exist', path_image)

    print(len(input_list), len(bbox_list))
    return input_list, bbox_list
    
input_list_pig, bbox_list_pig = load_pig()#1709, 1759

74 74


비용

In [27]:
boxes = np.concatenate(bbox_list_pig, 0)
cls = boxes[:, 0]
np.sum(cls > 3), boxes.shape

(65, (78, 5))

In [24]:
len(input_list_pig)

74

In [25]:
class_names_kor

['',
 '모름',
 '돼지',
 '멧돼지',
 '큰부리까마귀',
 '흰배지빠귀',
 '유리딱새',
 '동고비',
 '박새',
 '호랑지빠귀',
 '검독수리',
 '너구리',
 '설치류',
 '다람쥐',
 '사람']

In [42]:
np.sum(cls == class_names_kor.index('큰부리까마귀')), np.sum(cls != class_names_kor.index('큰부리까마귀')), len(input_list_pig)

(13, 65, 74)

In [40]:
cost_image = 20
cost_crow = 20
cost_crow_other = 100

In [44]:
cost_crow * np.sum(cls == class_names_kor.index('큰부리까마귀')) + cost_crow_other * np.sum(cls != class_names_kor.index('큰부리까마귀')) +  cost_image * len(input_list_pig)

8240

In [16]:
def crop_bbox_image(image, boxes):
    img_objects = []
    image = np.array(image)
    for box in boxes:        
        box = box.astype(np.int)
        x1, y1, x2, y2 = box
        w = x2 - x1
        h = y2 - y1        
        crop_image_arr = image[y1:y2, x1:x2]
        ch, cw, cc = crop_image_arr.shape
        if ch>1 and cw>1:
            img_objects.append(crop_image_arr)
        else:
            print('crop_bbox_image', x2-x1, y2-y1, 'crop_image_arr.shape', crop_image_arr.shape)
        
    return img_objects
    
    
def attach_crop_image(image, boxes, max_crop=200):
        
    crop_bbox_arr = crop_bbox_image(image, np.array(boxes)[:max_crop])
    bbox_k = len(crop_bbox_arr)
    max_col = 30
    
    if bbox_k > 0:
        img_h, img_w, img_c = image.shape
        object_img_w = img_w//bbox_k        
        resize_h = img_h // 8
        resize_w = img_w // bbox_k  
        resize_w = min(max(resize_w, img_w//max_col), img_w//8)
        
        footer_h = resize_h * (1 + (bbox_k-1)//max_col)
        footer = np.zeros((footer_h, img_w, img_c), np.uint8)
        
        for i in range(min(bbox_k, max_crop)):
            crop_arr = crop_bbox_arr[i]
            crop_img = Image.fromarray(crop_arr)                
            crop_img = crop_img.resize((resize_w, resize_h))
            crop_arr_resized = np.array(crop_img)
            offset_y = (i//max_col) * resize_h
            offset_x = (i%max_col) * resize_w
            footer[offset_y:offset_y+resize_h, offset_x:offset_x+resize_w] = crop_arr_resized

        seperate_line = np.zeros_like(footer[:2])
        image = np.concatenate((image, seperate_line, footer), axis=0)    
    return image    

In [17]:
def visualize_detections_simple(
    image, boxes, classes, figsize=(12, 12), linewidth=1, color=[0, 0, 1]
):
    """Visualize Detections"""
    image = np.array(image, dtype=np.uint8)    
    
    img_h, img_w, img_c = image.shape
    
    image = attach_crop_image(image, boxes, max_crop=100)
    
    plt.figure(figsize=figsize)
    plt.axis("off")
    plt.imshow(image)
    ax = plt.gca()
    boxes_width = boxes[:, 2] - boxes[:, 0]
    boxes_height = boxes[:, 3] - boxes[:, 1]
    box_min_width = np.min(boxes_width)
    box_max_width = np.max(boxes_width)
    title = str.format('(%dx%d) %d box, width:%d ~ %d' 
                       %(img_h, img_w, len(boxes), box_min_width, box_max_width))
    plt.title(title)
    for box, cls in zip(boxes, classes):
        x1, y1, x2, y2 = box        
        w, h = x2 - x1, y2 - y1
        color = edgecolors[int(cls)]
        patch = plt.Rectangle(
            [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth
        )
        ax.add_patch(patch)
        if len(boxes) < 70:
            score_txt = class_names[int(cls)]
            ax.text(x1, y1, score_txt, bbox={"facecolor": [1,1,0], "alpha": 0.4}, clip_box=ax.clipbox, clip_on=True,)
        
    plt.show()
    return ax

def display_data(X, BBOX, stride=1):
    for i in range(len(X)):
        if i%stride==0:
            img_arr = X[i]
            sample_box = BBOX[i]
            label = sample_box[:, 0]
            bbox = sample_box[:, 1:]

            h, w, c = img_arr.shape
            scale = np.array((w, h, w, h))
            scale = np.reshape(scale, (1, 4))
            bbox_norm = bbox.astype(np.float) * scale.astype(np.float)
            #print('bbox_norm', bbox, bbox_norm)
            print(i, np.unique(label))
            ax = visualize_detections_simple(img_arr,bbox_norm,label)

In [ ]:
display_data(input_list_pig, bbox_list_pig, stride=1)